# 📜 Cell 1 – Algorithm Overview & Pseudocode

**Flat TT‑KNN with Stay‑Prior & Dynamic Look‑Ahead (Humob 2025)**  

**Training**  
1. **Load & split**  ➜ days 1‑60 + ρ · unmasked(61‑75) ⇢ **train**, rest ⇢ **val**  
2. **Build per‑user statistics**  
   * `stay_p`  = P(locₜ₊₁ = locₜ)  
   * `median_gap` = median #segments between successive *moves*  
3. **Dynamic look‑ahead** `Mᵢ = clip(median_gap, 1, 6)`  
4. **Build transition table** `(segment, cur_loc) → Counter(next_loc)`  
   * Filter locations visited `< τ` times  
   * Keep transitions with `gap ≤ Mᵢ`

**Prediction (per user)**  
1. Let current `(d,t, loc₀)`  
2. Look ahead `1…Mᵢ` segments; collect candidate next locations  
3. **Score** each candidate  
   *   `norm_dist`   (lower better)  
   *   `norm_freq`   (higher better)  
4. Lowest blended score wins (ties ↔ top‑K)  
5. **Stay bias**: If predicted ≠ loc₀, choose `loc₀` instead when  
   `λ·stay_p > 1 − stay_p`

**Evaluation**  
* Sequential prediction across validation set  
* Per‑user **GeoBLEU** & **DTW**, then macro‑average.


# 🛠️ Cell 2 – Install external metrics package
!pip install -q git+https://github.com/yahoojapan/geobleu.git


In [ ]:
# 📚 Cell 3 – Imports & Global Constants
import numpy as np, pandas as pd, random, seaborn as sns, matplotlib.pyplot as plt
from tqdm.auto import tqdm
from collections import defaultdict, Counter
from geobleu import calc_geobleu_single, calc_dtw_single
import warnings, os, json, math
warnings.filterwarnings('ignore')

# --- reproducibility ---
np.random.seed(42)
random.seed(42)

# --- Dataset parameters (Humob 2025) ---
DATA_DIR       = "data"      # change if needed
MASK_VALUE     = 999
COLUMNS        = ["uid", "d", "t", "x", "y"]
DTYPES         = {"uid": "int32","d": "int8","t": "int8","x": "int16","y": "int16"}
DAY_TRAIN_MAX  = 60
DAY_VAL_MIN    = 61
DAY_VAL_MAX    = 75

# --- Base hyper‑params ---
BASE_CFG = dict(
    DELTA       = 30,     # minutes per segment
    TAU         = 5,      # min visits per location
    M_DEFAULT   = 2,      # fallback look‑ahead
    K           = 2,      # KNN
    FREQ_W      = 0.5,    # frequency weight in score
    STAY_W      = 0.4,    # λ for stay bias
    TRAIN_FRAC  = 0.7     # ρ  (train share in days 61‑75)
)

print("Globals ready.")


Globals ready.


In [ ]:
# 🔧 Cell 4 – Distance & Time‑Segment Utilities
def euclidean(a, b):
    return math.hypot(a[0]-b[0], a[1]-b[1])

def manhattan(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def chebyshev(a, b):
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

def calc_dist(a, b, typ='euclidean'):
    return {'euclidean': euclidean,
            'manhattan': manhattan,
            'chebyshev': chebyshev}[typ](a, b)

def to_flat_segment(day, t, delta=30):
    """Return global segment index since day‑0 00:00"""
    segs_per_day = (24*60) // delta
    return day*segs_per_day + (t*60)//delta


In [ ]:
# 📈 Cell 5 – Per‑User Statistics: stay_p and median_gap
def compute_user_stats(traj, delta):
    """
    traj : list[((x,y),(d,t))] chronological
    → (stay_probability, median_gap_segments)
    """
    if len(traj) < 2:
        return 0.0, 1
    traj = sorted(traj, key=lambda z: (z[1][0], z[1][1]))
    stay, gaps = 0, []
    last_seg = to_flat_segment(*traj[0][1], delta)
    last_loc = traj[0][0]
    for loc, (d, t) in traj[1:]:
        seg = to_flat_segment(d, t, delta)
        gap = seg - last_seg
        if gap <= 0:          # duplicate or unordered
            continue
        if loc == last_loc:
            stay += 1
        else:
            gaps.append(gap)
            last_loc = loc
        last_seg = seg
    median_gap = int(np.median(gaps)) if gaps else 1
    return stay / max(len(traj)-1,1), median_gap


In [ ]:
# 🏗️ Cell 6 – Transition‑Table Builder (accepts user‑specific M)
def build_flat_TT_index(trajectory, tau=5, delta=30, M=2):
    """
    trajectory : list[(d, t, x, y)]
    returns    : TT_index, TT_freq  (defaultdict structures)
    """
    loc_counts = Counter((x,y) for _,_,x,y in trajectory)
    traj_filt  = [(d,t,x,y) for d,t,x,y in trajectory if loc_counts[(x,y)] >= tau]
    TT_idx = defaultdict(lambda: defaultdict(list))
    TT_freq= defaultdict(lambda: defaultdict(Counter))
    seg_traj = [(to_flat_segment(d,t,delta),(x,y)) for d,t,x,y in traj_filt]
    seg_traj.sort()
    for i in range(len(seg_traj)-1):
        seg1, loc1 = seg_traj[i]
        seg2, loc2 = seg_traj[i+1]
        if 0 < seg2-seg1 <= M:
            TT_idx[seg1][loc1].append(loc2)
            TT_freq[seg1][loc1][loc2] += 1
    return TT_idx, TT_freq


In [ ]:
# 🤖 Cell 7 – FlatTTKNNModel++  (stay prior + dynamic look‑ahead)
class FlatTTKNNModel:
    def __init__(self,
                 tau           = BASE_CFG['TAU'],
                 delta         = BASE_CFG['DELTA'],
                 M_default     = BASE_CFG['M_DEFAULT'],
                 K             = BASE_CFG['K'],
                 distance_type = 'euclidean',
                 freq_weight   = BASE_CFG['FREQ_W'],
                 stay_weight   = BASE_CFG['STAY_W']):
        self.tau, self.delta = tau, delta
        self.M_default, self.K = M_default, K
        self.distance_type = distance_type
        self.freq_weight   = freq_weight
        self.stay_weight   = stay_weight
        self.index, self.freq_index = {}, {}
        self.user_M, self.user_stay_p = {}, {}

    # ---------- fit ----------
    def fit(self, user_trajs):
        for uid, traj in tqdm(user_trajs.items(), desc="Build indices", leave=False):
            stay_p, med_gap = compute_user_stats(traj, self.delta)
            self.user_stay_p[uid] = stay_p
            self.user_M[uid]      = int(np.clip(med_gap, 1, 6))
            formatted = [(d,t,x,y) for (x,y),(d,t) in traj]
            if formatted:
                idx,freq = build_flat_TT_index(formatted,
                                               tau   = self.tau,
                                               delta = self.delta,
                                               M     = self.user_M[uid])
                self.index[uid], self.freq_index[uid] = idx, freq

    # ---------- core predictor ----------
    def _predict_knn(self, uid, d, t, loc):
        """plain TT‑KNN w/out stay bias"""
        if uid not in self.index:
            return loc
        M = self.user_M.get(uid, self.M_default)
        curr_seg = to_flat_segment(d,t,self.delta)
        cand, cand_freq = [], []
        for o in range(1, M+1):
            seg = curr_seg + o
            if seg in self.index[uid] and loc in self.index[uid][seg]:
                nxts = self.index[uid][seg][loc]
                freqs= [self.freq_index[uid][seg][loc][n] for n in nxts]
                cand.extend(nxts); cand_freq.extend(freqs)
        if not cand:
            return loc
        uniq = {}
        for l,f in zip(cand,cand_freq):
            uniq[l] = uniq.get(l,0)+f
        dist_vals = [calc_dist(loc,l,self.distance_type) for l in uniq]
        min_d,max_d = min(dist_vals), max(dist_vals)
        rng = max(max_d-min_d,1e-6)
        best = sorted(
            (( ( (calc_dist(loc,l,self.distance_type)-min_d)/rng ) /
               ( (uniq[l]/max(uniq.values()))**self.freq_weight + 1e-6 ),
               l) for l in uniq),
            key=lambda z:z[0])[:self.K]
        for _,l in best:
            if l!=loc: return l
        return loc

    # ---------- public predict ----------
    def predict(self, uid, d, t, loc):
        raw_pred = self._predict_knn(uid,d,t,loc)
        stay_p   = self.user_stay_p.get(uid,0)
        if raw_pred!=loc and (self.stay_weight*stay_p > (1-stay_p)):
            return loc
        return raw_pred


In [ ]:
# 🗂️ Cell 8 – Dummy Baseline Predictors
class DummyPredictor:
    def __init__(self, strategy='fixed', fixed_loc=(0,0)):
        self.strategy, self.fixed_loc = strategy, fixed_loc
        self.common = {}
    def fit(self, user_trajs):
        if self.strategy=='random':
            for u,tr in user_trajs.items():
                locs=[l for l,_ in tr]
                cnt=Counter(locs)
                self.common[u]=[l for l,_ in cnt.most_common(5)] or [locs[0]]
    def predict(self, uid,d,t,loc):
        if self.strategy=='fixed': return self.fixed_loc
        if self.strategy=='last' : return loc
        if self.strategy=='random':
            return random.choice(self.common.get(uid,[loc]))
        return loc


In [ ]:
# 📂 Cell 9 – Data Loading & Splitting (train / val)
def load_and_split(city, train_frac=BASE_CFG['TRAIN_FRAC'], seed=42):
    path = f"{DATA_DIR}/city_{city}_challengedata.csv"
    train_early, tail = [], []
    for chunk in tqdm(pd.read_csv(path,usecols=COLUMNS,dtype=DTYPES,chunksize=500_000)):
        te = chunk[(chunk.d<=DAY_TRAIN_MAX)&(chunk.x!=MASK_VALUE)]
        tl = chunk[(chunk.d>=DAY_VAL_MIN)&(chunk.d<=DAY_VAL_MAX)&(chunk.x!=MASK_VALUE)]
        if not te.empty: train_early.append(te)
        if not tl.empty: tail.append(tl)
    train_df = pd.concat(train_early) if train_early else pd.DataFrame(columns=COLUMNS)
    tail_df  = pd.concat(tail) if tail else pd.DataFrame(columns=COLUMNS)
    # split tail by users
    np.random.seed(seed)
    users = tail_df.uid.unique()
    n_tr  = int(len(users)*train_frac)
    tr_users = set(np.random.choice(users, n_tr, replace=False))
    train_tail = tail_df[tail_df.uid.isin(tr_users)]
    val_df     = tail_df[~tail_df.uid.isin(tr_users)]
    full_train = pd.concat([train_df, train_tail], ignore_index=True)
    print(f"Loaded City {city}: train={len(full_train):,}, val={len(val_df):,}")
    return full_train, val_df


In [ ]:
# 🔄 Cell 10 – Helper: dataframe ➜ user trajectories dictionary
def df_to_trajs(df):
    trajs = defaultdict(list)
    df_s  = df.sort_values(['uid','d','t'])
    for uid,grp in df_s.groupby('uid'):
        locs = list(zip(grp.x,grp.y))
        ts   = list(zip(grp.d,grp.t))
        trajs[uid] = list(zip(locs, ts))
    return dict(trajs)


In [ ]:
# 🔮 Cell 11 – Sequential Prediction Loop
def run_prediction(model, val_df, hist_trajs):
    val_sorted = val_df.sort_values(['uid','d','t']).copy()
    out = val_sorted[['uid','d','t','x','y']].copy()
    out['x_pred']=np.nan; out['y_pred']=np.nan
    for uid,grp in tqdm(val_sorted.groupby('uid'), desc="Predict"):
        history = list(hist_trajs.get(uid, []))
        preds=[]
        for _,row in grp.iterrows():
            d,t,row_loc = row.d,row.t,(row.x,row.y)
            last_loc = history[-1][0] if history else row_loc
            pred = model.predict(uid,d,t,last_loc)
            preds.append(pred)
            history.append((pred,(d,t)))         # update with prediction
        idx = grp.index
        out.loc[idx,'x_pred']=[p[0] for p in preds]
        out.loc[idx,'y_pred']=[p[1] for p in preds]
    return out


In [ ]:
# 📏 Cell 12 – Metric Calculator (GeoBLEU & DTW)
def eval_metrics(pred_df, val_df):
    merged = pred_df.merge(
        val_df.rename(columns={'x':'x_gt','y':'y_gt'}),
        on=['uid','d','t'], how='inner')
    if merged.empty:
        return dict(geobleu=0., dtw=float('inf'))
    met=[]
    for uid,g in merged.groupby('uid'):
        if len(g)<2: continue
        gt  = [(int(r.d),int(r.t),int(r.x_gt),int(r.y_gt)) for _,r in g.sort_values(['d','t']).iterrows()]
        pr  = [(int(r.d),int(r.t),int(r.x_pred),int(r.y_pred)) for _,r in g.sort_values(['d','t']).iterrows()]
        try:
            met.append((calc_geobleu_single(pr,gt), calc_dtw_single(pr,gt)))
        except Exception: pass
    if not met: return dict(geobleu=0., dtw=float('inf'))
    arr=np.array(met)
    return dict(geobleu=arr[:,0].mean().round(5), dtw=arr[:,1].mean().round(3))


In [ ]:
# 🔍 Cell 13 – Hyper‑parameter Grid Search (optional narrow grid)
GRID = [
    dict(distance_type='euclidean', freq_weight=0.5, stay_weight=0.4),
    dict(distance_type='manhattan', freq_weight=1.0, stay_weight=0.4),
    dict(distance_type='chebyshev',  freq_weight=0.5, stay_weight=0.6)
]
def run_search(city):
    tr_df, val_df = load_and_split(city)
    if tr_df.empty or val_df.empty: return None
    tr_trajs = df_to_trajs(tr_df)
    best, best_score = None, -1
    for i,cfg in enumerate(GRID,1):
        print(f"\nConfig {i}/{len(GRID)} ➜", json.dumps(cfg))
        model = FlatTTKNNModel(**cfg)
        model.fit(tr_trajs)
        pred_df = run_prediction(model, val_df, tr_trajs)
        scores = eval_metrics(pred_df, val_df)
        print("  GeoBLEU",scores['geobleu'],"DTW",scores['dtw'])
        if scores['geobleu']>best_score:
            best_score,best = scores['geobleu'], dict(cfg, **scores)
    print("\n🏆 Best:", best)
    return best


In [ ]:
# 🚀 Cell 14 – Execute for target city/cities
CITIES = ["F"]           # change / extend
results = {}
for city in CITIES:
    print(f"\n=== City {city} ===")
    results[city] = run_search(city)
print("\n🎉 DONE")



=== City F ===


0it [00:00, ?it/s]

Loaded City F: train=471, val=29

Config 1/3 ➜ {"distance_type": "euclidean", "freq_weight": 0.5, "stay_weight": 0}


Build indices:   0%|          | 0/25 [00:00<?, ?it/s]

Predict:   0%|          | 0/8 [00:00<?, ?it/s]

...

Predict:   0%|          | 0/8 [00:00<?, ?it/s]

  GeoBLEU 0.0006 DTW 27.09

Config 3/3 ➜ {"distance_type": "chebyshev", "freq_weight": 0.5, "stay_weight": 0}


Build indices:   0%|          | 0/25 [00:00<?, ?it/s]

Predict:   0%|          | 0/8 [00:00<?, ?it/s]

  GeoBLEU 0.0006 DTW 27.09

🏆 Best: {'distance_type': 'euclidean', 'freq_weight': 0.5, 'stay_weight': 0, 'geobleu': 0.0006, 'dtw': 27.09}

🎉 DONE
